In [1]:
import numpy as np
#from sklearn.datasets import load_iris, load_digits
#from sklearn.model_selection import train_test_split
import pandas as pd
import altair as alt
import umap

In [2]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [ ]:
alt.renderers.enable('notebook')

## UMAP redction of gametocyte data

### Load and Clean Data

iris = load_iris()
test_df = pd.DataFrame(data=iris.data)
embedding = reducer.fit_transform(iris.data)
embedding.shape
test_df['x'] = embedding[:, 0]
test_df['y'] = embedding[:, 1]
test_df['label'] = iris.target
test_df
test_df.columns = ['One', 'Two', 'Three', 'Four', 'x', 'y', 'label']

In [ ]:
data = pd.read_csv('../data/raw_data/rpkm_kent2018.tsv.gz', sep='\t')
data.head()

In [ ]:
#data.set_index(['sample_id', 'gene'], inplace=True)

pivot_data = data.pivot(index='sample_id', columns='gene_id')['logRPKM']
pivot_data.head()

In [ ]:
reducer = umap.UMAP()

pivot_data.describe()

In [ ]:
test_em = reducer.fit_transform(pivot_data)
#trans_test = umap.UMAP(n_neighbors=5, n_components=2,).fit(pivot_data) #min_dist=0

In [ ]:
test_em[:, 0]
test_em[:, 1]

In [ ]:
samples = pd.read_csv('sample_sheet.tsv', sep='\t')
samples.sort_values(by=['stage'])

In [ ]:
results = pd.DataFrame({
    # testing new near neighbour value.
    'x': trans_test.embedding_[:, 0],
    'y': trans_test.embedding_[:, 1], 
#     'x': test_em[:, 0],
#     'y': test_em[:, 1], 
    'sample_id': pivot_data.index
})


results = results.join(samples.set_index('sample_id'), on='sample_id')
results_all = results.join(pivot_data, on='sample_id')
results
 

In [ ]:
results_all.head()
melt_results = results_all.melt(id_vars=['sample_id', 'rapa_treat', 'x',  'y', 'stage'], value_vars=['strand'])

melt_results = melt_results.drop(['value', 'variable'], axis=1)
melt_results

#pd.melt(df, id_vars=['A'], value_vars=['B', 'C'])

In [ ]:
final_data = data.set_index('sample_id').join(melt_results.set_index('sample_id'), on='sample_id')
final_data

In [ ]:
# final_data.to_csv('rpkm_data_umap_coords', sep='\t')

both_data = final_data[['x', 'y', 'logRPKM', 'gene_id']]
both_data.head()
both_data.to_json(orient='records', path_or_buf='both_data.json')

In [ ]:
scale=alt.Scale(domain=(6, 13)) 

no_rapa = alt.Chart(melt_results[melt_results['rapa_treat']=='no']).mark_point(size=60).encode( #[final_data['rapa_treat']=='no']
    alt.X('x', axis=alt.Axis(title=''), type='quantitative'),
    alt.Y('y', axis=alt.Axis(title=''), type='quantitative', scale=scale),
    tooltip = ['rapa_treat', 'stage'],
    color = 'stage:N'
)

rapa = alt.Chart(melt_results[melt_results['rapa_treat']=='yes']).mark_circle(size=60).encode(
    alt.X('x:Q'),
    alt.Y('y:Q', scale=scale),
    tooltip = ['rapa_treat', 'stage'],
    color = 'stage:N'
)

#input_dropdown = alt.binding_select(options=final_data['gene_id'].unique().tolist())
input_dropdown = alt.binding_select(options=['PBANKA_0000201', 'PBANKA_0000301'])
selection = alt.selection_single(fields=['gene_id'], bind=input_dropdown, name='Gene ID', clear='false')

# genre_dropdown = alt.binding_select(options=genres)
# genre_select = alt.selection_single(fields=['Major_Genre'], bind=genre_dropdown, name="Genre")


#both = alt.Chart('both_data.json').mark_circle(size=60).encode(
#both = alt.Chart(final_data[final_data['gene_id'].isin(['PBANKA_0000201', 'PBANKA_0000301'])]).mark_circle(size=60).encode(
both = alt.Chart(both_data).mark_circle(size=60).encode(   
    alt.X('x:Q', axis=alt.Axis(title='')), 
    alt.Y('y:Q', axis=alt.Axis(title=''), scale=scale),
    tooltip = ['logRPKM:Q',], # 'rapa_treat:O', 'stage:O'],
    color = alt.Color('logRPKM:Q', scale=alt.Scale(scheme='redblue')),
#    stroke = alt.value('black'),    
).add_selection(
    selection
).transform_filter(
    selection
)

both_circles = alt.Chart(melt_results).mark_point(size=65).encode( #[final_data['rapa_treat']=='no']
    alt.X('x', axis=alt.Axis(title=''), type='quantitative'),
    alt.Y('y', axis=alt.Axis(title=''), type='quantitative', scale=scale),
    color = alt.value('grey'),
    # tooltip = ['rapa_treat', 'stage'],
)

#selection = alt.selection_single(fields=['Gene'], bind=input_dropdown, name=' ', clear='false')

combined = no_rapa + rapa | both_circles + both
combined

#TODO load from json for both plot to get rpkm values. 

In [ ]:
combined.save('combined.html')

In [ ]:
input_dropdown = alt.binding_select(options=final_data['gene_id'].unique().tolist())
selection = alt.selection_single(fields=['gene_id'], bind=input_dropdown, name=' ', clear='false')

# genre_dropdown = alt.binding_select(options=genres)
# genre_select = alt.selection_single(fields=['Major_Genre'], bind=genre_dropdown, name="Genre")

# scale=alt.Scale(domain=(6, 12)),

both = alt.Chart(final_data).mark_circle(size=60).encode(
    alt.X('x', axis=alt.Axis(title='')), 
    alt.Y('y', axis=alt.Axis(title='')),
    tooltip = 'logRPKM:Q',
    color = 'logRPKM:Q',
     
).add_selection(
    selection
).transform_filter(
    selection
)


both.save('single_coloured.html')

In [ ]:
genre_dropdown = alt.binding_select(options=genres)
genre_select = alt.selection_single(fields=['Major_Genre'], bind=genre_dropdown, name="Genre")

main.encode(color=alt.condition(selection, 'Gene Data:Q', alt.value('lightgray'))).add_selection(
selection
).transform_filter(
    selection
)

In [ ]:
# genes by cell type vector

In [ ]:
all_s = alt.Chart(melt_results).mark_point(size=60).encode( #[final_data['rapa_treat']=='no']
    alt.X('x', axis=alt.Axis(title=''), type='quantitative'),
    alt.Y('y', axis=alt.Axis(title=''), type='quantitative'),
    tooltip = 'sample_id:N',
    color = 'stage:N'
)

all_s 

In [ ]:
# umap of genes
genes_umap = pivot_data.T


In [ ]:
genes_umap.head()

In [ ]:
#test_em_genes = reducer.fit_transform(genes_umap)

trans = umap.UMAP(n_neighbors=20, n_components=2, min_dist=0).fit(genes_umap) 
# n_components

In [ ]:
# test_em[:, 0]
# test_em_genes[:, 1]

trans.embedding_[:, 1]

In [ ]:
# genes_umap['x'] = test_em_genes[:, 0]
# genes_umap['y'] = test_em_genes[:, 1]

genes_umap['x'] = trans.embedding_[:, 0]
genes_umap['y'] = trans.embedding_[:, 1]
# genes_umap['z'] = trans.embedding_[:, 2]


In [ ]:
genes_umap.head()

In [ ]:
genes_umap.to_csv('all_genes_umap' ,sep='\t')

In [ ]:
genes_umap.quantile(0.95)  #describe()

In [ ]:
genes_umap.columns

In [ ]:
genes_umap[['SRX3653212_06h_no_3', 'SRX3653213_24h_no_4', 'x']]

In [ ]:
umap_genes = alt.Chart(genes_umap.reset_index()).mark_circle(size=60).encode(
    alt.X('x:Q'),
    alt.Y('y:Q'),
    tooltip = ['gene_id:N'],
     #, bin=alt.Bin(maxbins=5))
).properties(
    height=600,
    width=600
).interactive()

# umap_genes_z = alt.Chart(genes_umap.reset_index()).mark_circle(size=60).encode(
#     alt.X('y:Q'),
#     alt.Y('z:Q'),
#     tooltip = ['gene_id:N'],
#      #, bin=alt.Bin(maxbins=5))
# ).properties(
#     height=600,
#     width=600
# ).interactive()

l = umap_genes.encode(color = alt.Color('SRX3653199_30h_yes_2:Q', scale=alt.Scale(scheme="redblue")))
# l_z = umap_genes.encode(color = alt.Color('SRX3653199_30h_yes_2:Q', scale=alt.Scale(scheme="redblue")))
r = umap_genes.encode(color = alt.Color('SRX3653182_00h_yes_1:Q', scale=alt.Scale(scheme="redblue")))

l | r

In [ ]:
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.layouts import layout, column
from bokeh.models import ColumnDataSource, Div, Whisker
from bokeh.models.widgets import Slider, Select, TextInput
from bokeh.io import output_notebook, show, output_file
from bokeh.models import HoverTool, Legend

import pandas as pd
import numpy as np


output_notebook()


In [ ]:
melt_results.head()

In [ ]:
colours = {'00h': '#005ab7', '06h': '#6a4c93', '12h': '#8ac926', '18h': '#ffca3a', '24h': '#fcab10', '30h': '#ff595e'}
melt_results['colours'] = melt_results['stage'].map(colours)
melt_results.to_csv('umap_data.tab', sep='\t')

In [ ]:
output_file("bokeh_plot.html")

In [ ]:
final_data.groupby(['x', 'y', 'rapa_treat']).size()

In [ ]:
# Colours for the data
colours = {'00h': '#005ab7', '06h': '#6a4c93', '12h': '#8ac926', '18h': '#ffca3a', '24h': '#fcab10', '30h': '#ff595e'}
melt_results['colours'] = melt_results['stage'].map(colours)


source_xy = ColumnDataSource(final_data[(final_data['gene_id'] == 'PBANKA_0000301')])

source_xy_rapa = ColumnDataSource(melt_results[melt_results['rapa_treat'] == 'yes'])
source_xy_norapa = ColumnDataSource(melt_results[melt_results['rapa_treat'] == 'no'])

# Tools for the charts
hover_xy = HoverTool(
        tooltips=[
            ("Rapamycin Treatment", "@rapa_treat"),
            ("Stage", "@stage"),
        ]
    )

hover_rpkm = HoverTool(
        tooltips=[
            ("Rapamycin Treatment", "@rapa_treat"),
            ("Stage", "@stage"),
        ]
    )

base_plot_options = dict(width=400, plot_height=400, tools=["box_select, pan, reset, wheel_zoom"])

xy_plot_options = base_plot_options
xy_plot_options['tools'].append(hover_xy)

rpkm_plot_options = base_plot_options
rpkm_plot_options['tools'].append(hover_rpkm)


# Plotting the data - xy from umap
xy_data = figure(**xy_plot_options)
xy_data.circle('x', 'y', size=10, source=source_xy_rapa, line_width=0, color='colours')
xy_data.circle('x', 'y', size=10, source=source_xy_norapa, fill_alpha=0, line_width=2, color='colours')

# rpkm color mapper
max_c = max(source_xy.data['logRPKM'])
min_c = min(source_xy.data['logRPKM'])
print(max_c, min_c)

from bokeh.models import LinearColorMapper, ColorBar
from bokeh.transform import linear_cmap 
from bokeh.palettes import RdBu
mapper = LinearColorMapper(palette='Viridis256', low=min_c, high=max_c)
#print(mapper)

# colormapper = linear_cmap(field_name = final_data[(final_data['gene_id'] == 'PBANKA_0000301')]['logRPKM'], 
#                           palette=RdBu, low=min_c, high=max_c)


# Plotting the data - rpkm
rpkm_data = figure(x_range=xy_data.x_range, y_range=xy_data.y_range ,**rpkm_plot_options, )
rpkm_data.circle('x', 'y', size=10, source=source_xy, line_color='grey', fill_color={'field': 'logRPKM', 'transform': mapper})

color_bar = ColorBar(color_mapper=mapper, width=8,  location=(0,0))
rpkm_data.add_layout(color_bar, 'right')

#geneid_input = TextInput(value="default", title="Label:")

# LinearColorMapper

# Putting plots together in a grid
p = gridplot([[xy_data, rpkm_data]])

show(p)

In [ ]:
output_file("bokeh_plot.html")



In [ ]:
data = pd.read_csv('all_genes_umap', sep='\t')
data.head()

In [ ]:
data[data['gene_id'] == 'PBANKA_0827100']

In [ ]:
data2 = pd.read_csv('rpkm_data_umap_coords', sep='\t')
data2.head()
data2.dtypes

In [ ]:
mean_data1 = data2
mean_data1['stage'] = mean_data1['stage'].str.replace('h', '').astype('int32')

rapa_map = {'yes': 'Rapamycin', 'no': 'No Rapamycin'}

mean_data1['rapa_treat'] = mean_data1['rapa_treat'].map(rapa_map)

In [ ]:
mean_data2 = mean_data1.groupby(['gene_id', 'stage', 'rapa_treat']).mean()
mean_data3 = mean_data1.groupby(['gene_id', 'stage', 'rapa_treat']).std()
mean_data3['std'] = mean_data3['logRPKM']
mean_data3 = mean_data3.drop(['logRPKM', 'x', 'y'], axis=1)



In [ ]:
new = mean_data2.join(mean_data3, how='right', lsuffix='_mean', rsuffix='_std')
new['up_std'] = new['logRPKM'] + mean_data3['std']
new['low_std'] = new['logRPKM'] - mean_data3['std']
new.to_csv('rpkm_data_mean_stddev', sep='\t')
new

In [ ]:
new.loc['PBANKA_0827100']

In [ ]:
np.mean([-2.686662, -2.686662, -1.902118, -2.686662])

In [ ]:

rapa_data = ColumnDataSource(mean_data2.loc['PBANKA_0000101', :, 'Rapamycin'].reset_index()) 
no_rapa_data = ColumnDataSource(mean_data2.loc['PBANKA_0000101', :, 'No Rapamycin'].reset_index())

p = figure(width=600, height=350, title='logRPKM Change Over Time')

p.xaxis.axis_label = 'Time (h)'
p.yaxis.axis_label = 'logRPKM'

rapa_l = p.line('stage', 'logRPKM', line_width=2, color='#ea2b1f', source=rapa_data, )
rapa_c = p.circle('stage', 'logRPKM', fill_color="white", size=8, color='#ea2b1f', source=rapa_data, )
no_rapa_l = p.line('stage', 'logRPKM', line_width=2, color='#5fa8d3', source=no_rapa_data, )
no_rapa_c = p.circle('stage', 'logRPKM', fill_color="white", size=8, color='#5fa8d3', source=no_rapa_data, )

legend = Legend(items=[
    ("Rapamycin"   , [rapa_l, rapa_c]),
    ("No Rapamycin" , [no_rapa_l, no_rapa_c]),


    ], location="center", orientation='horizontal')

p.add_layout(legend, 'below')


# w_r = Whisker(source=rapa_data, base='logRPKM', upper="std", lower="std", line_color='#ea2b1f',) #x_range_name='stage')
# w_r.upper_head.line_color = '#ea2b1f'
# w_r.lower_head.line_color = '#ea2b1f'

p.add_layout(w_r)
show(p)



In [ ]:
rapa_source_data.data['x'] = []
rapa_source_data.data['x']

In [ ]:
from random import random

from bokeh.layouts import row
from bokeh.models import CustomJS, ColumnDataSource
from bokeh.plotting import figure, output_file, show
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn

output_file("callback.html")

x = [random() for x in range(500)]
y = [random() for y in range(500)]

s1 = ColumnDataSource(data=dict(x=x, y=y))
p1 = figure(plot_width=400, plot_height=400, tools="lasso_select", title="Select Here")
p1.circle('x', 'y', source=s1, alpha=0.6)

s2 = ColumnDataSource(data=dict(x=[], y=[]))
p2 = figure(plot_width=400, plot_height=400, x_range=(0, 1), y_range=(0, 1),
            tools="", title="Watch Here")
p2.circle('x', 'y', source=s2, alpha=0.6)

s1.selected.js_on_change('indices', CustomJS(args=dict(s1=s1, s2=s2), code="""
        var inds = cb_obj.indices;
        var d1 = s1.data;
        var d2 = s2.data;
        d2['x'] = []
        d2['y'] = []
        for (var i = 0; i < inds.length; i++) {
            d2['x'].push(d1['x'][inds[i]])
            d2['y'].push(d1['y'][inds[i]])
        }
        s2.change.emit();
    """)
)


columns = [
        TableColumn(field="x", title="x",),
        TableColumn(field="y", title="y"),
    ]

data_table = DataTable(source=s2, columns=columns, width=400, height=280)



layout = row(p1, p2, data_table)

show(layout)

In [ ]:
# PBANKA_0827100